In [15]:
import pandas as pd
import numpy as np
from typing import *
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

In [16]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    # print(last_hidden.shape)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(r"Embedding_Model/gte_base")
model = AutoModel.from_pretrained(r"Embedding_Model/gte_base")

In [18]:
# val1=tokenizer.save_pretrained(r"Embedding_Model/gte_large")
# val2= model.save_pretrained(r"Embedding_Model/gte_large/")

In [19]:
data_input = pd.read_csv(r"Generated_Result/10k.csv", sep=";")

In [20]:
data_input.fillna("None", inplace=True)
data_input.head()

C:\Users\jqi22\AppData\Local\Temp\ipykernel_23860\922194924.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_input.fillna("None", inplace=True)


,project,risk,stakeholder,idx
0,Geological CO2 sequestration,Mitigation of anthropogenic carbon dioxide emi...,None,0.0
1,development and validation of a discrete event...,None,commercial vehicles passing through the area,1.0
2,Urban renaissance project in the area of Gazi,Pollution from the industrial use of Gazi|whic...,Residents of Gazi|environmental activists|loca...,2.0
3,simulation of changes in live carbon stocks in...,droughts,forest ecosystem|climate change,4.0
4,Ecological enhancement of coastal and marine i...,degraded or reduced habitat for native species,ecosystem and habitat value|this paper present...,5.0


In [21]:
stupid: Dict[str, List] = {}
for col in ["project", "risk", "stakeholder"]:
    temp = data_input.loc[:, col]
    blyat = []
    for tep_val in temp.values.tolist():
        if tep_val[0] != tep_val[0] or tep_val=="None": continue
        tep_val = tep_val.split("|")
        blyat = [*blyat, *tep_val]
    stupid[col] = blyat

# stupid_copy = {col: [val for val in data_input[col].str.split("|", expand=True).stack().dropna()]\
#            for col in ["Project", "Risk", "Stakeholder"]}

In [22]:
temp_project = [stupid["risk"][val: val+5] for val in range(0, len(stupid["risk"]), 5)]

In [23]:
att= tokenizer(temp_project[0], padding=True, truncation=True, return_tensors="pt")
out_test = model(**att)
print(out_test.last_hidden_state.shape, type(out_test), "\n", len(temp_project[0])\
      , sum([len(val) for val in temp_project[0]]), max([len(val) for val in temp_project[0]]))
# print(model.config)
embedds = average_pool(out_test.last_hidden_state, att["attention_mask"])
print(embedds.shape)

torch.Size([5, 13, 768]) <class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'> 
 5 173 52
torch.Size([5, 768])


In [24]:
stupid["stakeholder"][:5]

['commercial vehicles passing through the area',
 'Residents of Gazi',
 'environmental activists',
 'local government officials',
 'and business owners who will be impacted by the proposed urban renaissance project']

In [25]:
import math

def embedding_value(col, input_data, threshold=1000):
    output_project = []
    for tax in input_data:
        for ele in tax:
            # Tokenize the input texts
            batch_dict = tokenizer(ele, max_length=512, padding=True, truncation=True, return_tensors='pt')
            outputs = model(**batch_dict)
            embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
            output_project.append(embeddings)
            torch.save(torch.cat(output_project, axis=0), f"Embedding_Model/gte_large_embedding/10k_{col}.pt")
    # del output_project

In [21]:
columns = data_input.columns.to_list()
temp = ["risk", "project"]

threshold = 200
lens = len(stupid[col])
new_item_list = []
for item in range(0, math.ceil(lens/threshold)):
    new_item_list.append(stupid[col][:(item+1)*threshold] if (item+1)*threshold<lens-1 else lens-1)
    
for col in temp:
    embedding_value(col=col, threshold=200)

In [14]:
# beside stakeholder, rest of them shall be reloaded again

torch.save(torch.cat(output_project, axis=0), "Embedding_Model/gte_large_embedding/three_ouput1_risk.pt")

In [14]:
# import gc
# gc.collect()
torch.cuda.empty_cache()

In [13]:
full_emd, atten_mask = outputs.last_hidden_state, batch_dict['attention_mask']

In [14]:
print(full_emd.shape, atten_mask.shape, embeddings.shape)

torch.Size([4, 6, 768]) torch.Size([4, 6]) torch.Size([4, 768])


In [13]:
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())

[[69.65817260742188, 88.03556060791016, 68.79690551757812]]


In [1]:
import pandas as pd

In [7]:
d1 = pd.read_csv(r"Generated_Result/1k_fp.csv", sep=';')
d1.to_excel(r"Generated_Result/new_split_quota.xlsx", index=False)